In [332]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [333]:
df = pd.DataFrame()

In [334]:
N = 10000
np.random.seed(42)
df['X1'] = np.random.normal(0, 1, N)
df['X2'] = 2*df['X1'] + np.random.normal(0, 1, N) + 0
df['X3'] = df['X1']**2
df['X4'] = df['X1']**3
df['X5'] = np.abs(df['X1'])
df['X6'] = df['X1']**2 + np.random.normal(0, 0.5, N)

# Con sklearn

In [335]:
from sklearn.linear_model import LinearRegression

In [336]:
model = LinearRegression()

In [337]:
model.fit(df['X1'].values.reshape(-1, 1), df['X2'].values.reshape(-1, 1))

LinearRegression()

In [338]:
model.coef_, model.intercept_

(array([[1.99146746]]), array([0.01351583]))

# Con statsmodel

In [297]:
import statsmodels.formula.api as smf

In [298]:
formula = 'X2 ~ X1'
# endogenas vs exogenas
# Ordinary Least Squares
lm = smf.ols(formula=formula, data=df).fit()
print(lm.params)

lm.rsquared

Intercept    0.013516
X1           1.991467
dtype: float64


0.7994258937230349

In [299]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     X2   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                 3.985e+04
Date:                Wed, 16 Sep 2020   Prob (F-statistic):               0.00
Time:                        16:10:02   Log-Likelihood:                -14199.
No. Observations:               10000   AIC:                         2.840e+04
Df Residuals:                    9998   BIC:                         2.842e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0135      0.010      1.350      0.177      -0.006       0.033
X1             1.9915      0.010    199.622      0.000       1.972       2.011
==============================================================================
Omnibus:                        2.425   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.297   Jarque-Bera (JB):                2.478
Skew:                          -0.008   Prob(JB):                        0.290
Kurtosis:                       3.075   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Calculo de R-squared

In [317]:
lm.predict()

array([ 1.0027059 , -0.26183303,  1.30336648, ..., -1.39109948,
        1.00081684,  1.29679447])

In [319]:
predictions = lm.params['X1'] * df['X1'] + lm.params['Intercept']

In [301]:
error = predictions - df['X2']

In [302]:
error.mean(), error.std(ddof=2)

(-2.0019541580040822e-16, 1.0010236907114534)

In [303]:
R_square = (df['X2'].var()-error.var())/df['X2'].var()
R_square

0.7994258937230349

# Intervalos de confianza - https://www2.isye.gatech.edu/~yxie77/isye2028/lecture12.pdf

In [304]:
from scipy.stats import norm

In [305]:
Sxx = ((df['X1'] - df['X1'].mean())**2).sum()

In [306]:
Sxx

10068.360554106486

### Para pendiente

In [307]:
coef_std = (error.var(ddof=2)/Sxx)**0.5

In [308]:
# std_err en la tabla
coef_std

0.009976196069731437

In [309]:
# [0.025	0.975] en tabla
alpha = 0.025
norm.ppf(alpha, lm.params['X1'], coef_std), norm.ppf(1-alpha, lm.params['X1'], coef_std)

(1.9719144769051347, 2.011020446903902)

### Para intercept

In [310]:
# para intercept

intercept_std = (error.var(ddof=2) * (1/len(df) + ((df['X1'].mean())**2)/Sxx))**0.5
lm.params['Intercept'], intercept_std

(0.01351582770542467, 0.010010259587521457)

In [311]:
norm.ppf(alpha, lm.params['Intercept'], intercept_std), norm.ppf(1-alpha, lm.params['Intercept'], intercept_std)

(-0.006103920562014168, 0.033135575972863505)

# P-values

### Para pendiente

In [312]:
# t
t_coef = lm.params['X1'] - 0/coef_std
t_coef

1.9914674619045183

### Para intercept

In [313]:
# t
t_intercept = (lm.params['Intercept'] - 0)/intercept_std
t_intercept

1.3501975235760286

In [314]:
(1-norm.cdf(t_intercept))*2

0.17695263234531322

In [315]:
norm.cdf(0, lm.params['Intercept'], intercept_std)*2

0.17695263234531322

# BIC

In [328]:
likelihood = np.log(norm.pdf(df['X2'], loc=predictions, scale=error.std(ddof=2))).sum()

In [331]:
BIC = -2.0 * likelihood + np.log(N) * 2

BIC

28415.654686786052

# Feature engineering

In [270]:
formula = 'X3 ~ X1'
lm = smf.ols(formula=formula, data=df).fit()
print(lm.params)

lm.rsquared

Intercept    1.006836
X1          -0.002302
dtype: float64


2.596250281539092e-06

In [271]:
formula = 'X3 ~ np.square(X1)'
lm = smf.ols(formula=formula, data=df).fit()
print(lm.params)

lm.rsquared

Intercept       -4.510281e-16
np.square(X1)    1.000000e+00
dtype: float64


1.0